In [1]:
import matplotlib.pyplot as plt
import numpy as np            
import pandas as pd        

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

#Đường dẫn thư mục train và test
train_path = r'.\train'
test_path = r'.\test'
val_path = r'.\val'

batch_size = 16 

img_height = 500
img_width = 500

In [2]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

test = test_data_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb', 
    shuffle=False,
    class_mode='binary',
    batch_size=batch_size
)

validation_generator = image_gen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

Found 1943 images belonging to 2 classes.
Found 611 images belonging to 2 classes.
Found 497 images belonging to 2 classes.


In [4]:
base_model = VGG16(include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
cnn = Model(inputs=base_model.input, outputs=predictions)
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.3, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

# Compute class weights
from sklearn.utils.class_weight import compute_sample_weight

unique_classes = np.unique(train_generator.classes)
weights = compute_sample_weight(class_weight='balanced', y=train_generator.classes)
cw = dict(zip(unique_classes, weights))
print(cw)

# Train the model
cnn.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    class_weight=cw,
    callbacks=callbacks_list
)

{0: 1.763157894736842, 1: 1.763157894736842}
Epoch 1/25
122/122 [==============================] - 1109s 9s/step - loss: 1.8317 - accuracy: 0.6444 - val_loss: 0.6785 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 2/25
122/122 [==============================] - 864s 7s/step - loss: 1.1248 - accuracy: 0.6830 - val_loss: 0.6272 - val_accuracy: 0.7022 - lr: 0.0010
Epoch 3/25
122/122 [==============================] - 842s 7s/step - loss: 1.0728 - accuracy: 0.6999 - val_loss: 0.6102 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 4/25
122/122 [==============================] - 841s 7s/step - loss: 1.0199 - accuracy: 0.7200 - val_loss: 0.6274 - val_accuracy: 0.7223 - lr: 0.0010
Epoch 5/25
122/122 [==============================] - ETA: 0s - loss: 1.0347 - accuracy: 0.7216
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
122/122 [==============================] - 841s 7s/step - loss: 1.0347 - accuracy: 0.7216 - val_loss: 0.6323 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 6/25
12

In [5]:
cnn.save('vgg16_1.h5')

c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model on the test data
test_accu = cnn.evaluate(test)
print('The testing accuracy is:', test_accu[1] * 100, '%')

# Predict the labels for the test data
y_pred = cnn.predict(test)
y_pred = np.round(y_pred).astype(int)  # Chuyển đổi dự đoán thành nhãn nhị phân (0 hoặc 1)

# Get the true labels
y_true = test.classes

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=['Normal', 'Disease'])
print('Classification Report:')
print(class_report)

# Extract Sensitivity, Specificity, and F1-Score from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity  # Recall là tên khác của Sensitivity
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Sensitivity: {sensitivity * 100:.2f}%')
print(f'Specificity: {specificity * 100:.2f}%')
print(f'F1-Score: {f1_score:.2f}')

39/39 [==============================] - 207s 5s/step - loss: 0.5277 - accuracy: 0.7398
The testing accuracy is: 73.97708892822266 %
39/39 [==============================] - 207s 5s/step
Confusion Matrix:
[[ 56 141]
 [ 18 396]]
Classification Report:
              precision    recall  f1-score   support

      Normal       0.76      0.28      0.41       197
     Disease       0.74      0.96      0.83       414

    accuracy                           0.74       611
   macro avg       0.75      0.62      0.62       611
weighted avg       0.74      0.74      0.70       611

Sensitivity: 95.65%
Specificity: 28.43%
F1-Score: 0.83
